## Import dependencies

In [1]:
import requests
import bs4 as bs
import re
import pandas as pd

## Define State Dictionary for iteration

In [2]:
state_list= ["Alabama","Alaska","Arizona","Arkansas","California","Colorado","Connecticut","Delaware","Florida","Georgia",
"Hawaii" ,"Idaho" ,"Illinois" ,"Indiana" ,"Iowa" ,"Kansas" ,"Kentucky" ,"Louisiana" ,"Maine" ,"Maryland" ,"Massachusetts", 
"Michigan" ,"Minnesota" ,"Mississippi" ,"Missouri" ,"Montana" ,"Nebraska" ,"Nevada" ,"New Hampshire" ,"New Jersey" ,
"New Mexico" ,"New York" ,"North Carolina" ,"North Dakota" ,"Ohio" ,"Oklahoma" ,"Oregon" ,"Pennsylvania" ,"Rhode Island" ,
"South Carolina" ,"South Dakota" ,"Tennessee" ,"Texas" ,"Utah" ,"Vermont" ,"Virginia" ,"Washington" ,"West Virginia" ,
"Wisconsin" ,"Wyoming"]

state_list_test= ["Alabama","Alaska","North Carolina","South Dakota","Colorado"]

## Iterate the request for each state

In [3]:
# for state in state_dict.keys():
#     response= requests.post('https://www.brewersassociation.org/wp-admin/admin-ajax.php',
#                  data= {
#                      "action": "get_breweries",
#                     "_id": state,
#                     "search_by": "statename"
#                  })
#     state_dict[state]= response

response_list= []

for state in state_list_test:
    response= requests.post('https://www.brewersassociation.org/wp-admin/admin-ajax.php',
                 data= {
                     "action": "get_breweries",
                    "_id": state,
                    "search_by": "statename"
                 })
    if response.status_code == 200:
        response_list.append(response)
        print(f"{state} found!")
    else:
        print(f"Not found. Skipping {state}...")

print("/\\"*30)
print("REQUEST COMPLETE")

Alabama found!
Alaska found!
North Carolina found!
South Dakota found!
Colorado found!
/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\
REQUEST COMPLETE


## Convert to BeautifulSoup object and prettify for easier reading

In [4]:
soup_list= [bs.BeautifulSoup(i.text,'html.parser') for i in response_list]

## Locate all the brewery classes

In [5]:
breweries= [soup.find_all(attrs= {"class":"brewery"}) for soup in soup_list]

In [6]:
for i in breweries:
    print(len(i))

50
44
329
35
468


## Locate the names and addresses in each brewery class

In [7]:
breweries_name= [[i.find_all(attrs={"class":"name"})[0].string for i in state] for state in breweries]

for i in breweries_name:
    print(i[0:5])

['5 Rivers Brewing LLC', 'Avondale Brewing Co', 'Back Forty Beer Co', 'Back Forty Beer Company - Birmingham', 'Below the Radar Brewing Co']
['49th State Brewing Co', '49th State Brewing Co - Anchorage', 'Alaskan Brewing Co.', 'Anchorage Brewing Co', 'Arkose Brewery']
['1323 R & D', '1718 Ocracoke Brewing', '217 Brew Works', '34 Degree North Experiment Station', '3rd Degree Brewhouse']
['A Homestead Brew', "Ben's Brewing Co", 'Big Sioux Brewing Company', 'Brewery In Planning - Piedmont', 'Brewery in Planning - Sioux Falls, SD']
['10 Barrel Brewing Co - Denver', '105 West Brewing Co', '12Degree Brewing', '14er Brewing Company', '1876 Ale Works, LLC']


In [8]:
breweries_address= [[i.find_all(attrs={"class":"address"})[0].string.rstrip() for i in state] for state in breweries]
    
for i in breweries_address:
    print(i[0:5])

['', '201 41st St S', '200 N 6th St', '3201 1st Avenue N', '220 Holmes Ave NE']
['248 Parks Hwy 5 Mile', '717 W 3rd Ave', '5429 Shaune Dr', '148 W 91st Ave', '650 E Steel Loop']
['1323 Capital Blvd 1323 R and D', '1129 Irvin Garrish Hwy', '217 South St S', '4802 Main St', '1625 N Main St']
['26685 486th Ave', '222 W 3rd St', '', '', '']
['2620 Walnut St', '1043 Park St', '820 Main St', '3120 Blake St', '3352 Templeton Gap Rd']


In [9]:
city_state_zip= [[i.find_all('li')[2] for i in state] for state in breweries]

p= re.compile("<li>(.*?), ([A-Z][A-Z]) (\d{5})?")

# city= [re.search(p, str(i)).group(1) for i in city_state_zip[4]]
city= [[re.search(p, str(i)).group(1) for i in state] for state in city_state_zip]
state= [[re.search(p, str(i)).group(2) for i in state] for state in city_state_zip]
zipcode= [[re.search(p, str(i)).group(3) for i in state] for state in city_state_zip]

for i in city:
    print(i[0:5])
for i in state:
    print(i[0:5])
for i in zipcode:
    print(i[0:5])

['Spanish Fort', 'Birmingham', 'Gadsden', 'Birmingham', 'Huntsville']
['Healy', 'Anchorage', 'Juneau', 'Anchorage', 'Palmer']
['Raleigh', 'Ocracoke', 'Wilson', 'Shallotte', 'Fuquay Varina']
['Valley Springs', 'Yankton', 'Humboldt', 'Piedmont', 'Sioux Falls']
['Denver', 'Castle Rock', 'Louisville', 'Denver', 'Colorado Springs']
['AL', 'AL', 'AL', 'AL', 'AL']
['AK', 'AK', 'AK', 'AK', 'AK']
['NC', 'NC', 'NC', 'NC', 'NC']
['SD', 'SD', 'SD', 'SD', 'SD']
['CO', 'CO', 'CO', 'CO', 'CO']
['36527', '35222', '35901', '35222', '35801']
['99743', '99501', '99801', '99515', '99645']
['27603', '27960', '27893', '28470', '27526']
['57068', '57078', '57035', '57769', '57103']
['80205', '80109', '80027', '80205', '80907']


## Convert to DataFrame

Having issues with this step right now because I made lists of lists... like a chump...

In [10]:
# states_df= pd.DataFrame({
#     "Brewery": [i for i in breweries_name],
#     "Address": breweries_address,
#     "City": city,
#     "State": state,
#     "ZIP Code": zipcode
# })

# states_df.head()